In [1]:
import numpy as np 
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

In [2]:
# 1/ Training data :

column_names = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']
raw_dataset = pd.read_csv('./data/training_dataset.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
train_dataset = raw_dataset.copy()
train_dataset.isna().sum()
train_dataset = train_dataset.dropna()
train_dataset.tail()

,Model_name,input_shape,memory,input_width,input_size,nb_layers,sum_activations,params
149,basic_model_350_45,"(1, 350, 350, 3)",3212.0,350,367500,77,24032836,28087872
150,basic_model_224_24,"(1, 224, 224, 3)",3132.0,224,150528,44,18477760,49420384
151,basic_model_240_26,"(1, 240, 240, 3)",2999.0,240,172800,44,335053672,36989976
152,basic_model_512_28,"(1, 512, 512, 3)",2836.0,512,786432,54,383584308,34196142
153,basic_model_224_33,"(1, 224, 224, 3)",2828.0,224,150528,60,31316236,22456930


In [32]:
# 1/ Test data :

column_names = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']
raw_dataset = pd.read_csv('./data/test_dim1.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset = raw_dataset.copy()
test_dataset.isna().sum()
test_dataset = test_dataset.dropna()
test_dataset.tail()

,Model_name,input_shape,memory,input_width,input_size,nb_layers,sum_activations,params
75,MNASNet1.0,"(512, 512, 3)",1565.0,512,786432,153,85346408,4251760
76,MNASNet1.0,"(720, 720, 3)",1752.5,720,1555200,153,169583288,4251760
77,EfficientNet-B4,"(90, 90, 3)",2540.0,90,24300,470,10284484,19466816
78,EfficientNet-B4,"(512, 512, 3)",2688.0,512,786432,470,305708296,19466816
79,EfficientNet-B4,"(720, 720, 3)",2668.0,720,1555200,470,607426440,19466816


In [4]:
train_dataset.pop('input_width')

0       448
1      1792
2       299
3      1792
4       299
       ... 
149     350
150     224
151     240
152     512
153     224
Name: input_width, Length: 154, dtype: int64

In [33]:
test_dataset.pop('input_width')

0      90
1     512
2     720
3      90
4     512
     ... 
75    512
76    720
77     90
78    512
79    720
Name: input_width, Length: 80, dtype: int64

In [6]:
Y = train_dataset.pop('memory')
X = train_dataset
Y_f = Y
X_f = X

In [7]:
X.tail()

,Model_name,input_shape,input_size,nb_layers,sum_activations,params
149,basic_model_350_45,"(1, 350, 350, 3)",367500,77,24032836,28087872
150,basic_model_224_24,"(1, 224, 224, 3)",150528,44,18477760,49420384
151,basic_model_240_26,"(1, 240, 240, 3)",172800,44,335053672,36989976
152,basic_model_512_28,"(1, 512, 512, 3)",786432,54,383584308,34196142
153,basic_model_224_33,"(1, 224, 224, 3)",150528,60,31316236,22456930


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.001, random_state=42, shuffle=True)

In [9]:
X_train_model_names = X_train.pop('Model_name')
X_train_input_shapes = X_train.pop('input_shape')

In [10]:
X_test_model_names = X_test.pop('Model_name')
X_test_input_shapes = X_test.pop('input_shape')

In [11]:
X_f_model_names = X_f.pop('Model_name')
X_f_input_shapes = X_f.pop('input_shape')
dtrain_f = xgb.DMatrix(X_f, label=Y_f)

In [12]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [34]:
Y_e = test_dataset.pop('memory')
X_e = test_dataset
X_e.tail()
X_test_e_model_names = X_e.pop('Model_name')
X_test_e_input_shapes = X_e.pop('input_shape')
dtest_e = xgb.DMatrix(X_e, label=Y_e)

In [14]:
params = {
    # Parameters that we are going to tune.
    'max_depth':15,
    'min_child_weight': 5,
    'eta':.08,
    'subsample': .8,
    'colsample_bytree': 1,
    'gamma' : .9,
    # Other parameters
    'objective':'reg:squarederror',
}

In [15]:
params['eval_metric'] = "rmse"
num_boost_round = 3000

In [16]:
print(params)

{'max_depth': 15, 'min_child_weight': 5, 'eta': 0.08, 'subsample': 0.8, 'colsample_bytree': 1, 'gamma': 0.9, 'objective': 'reg:squarederror', 'eval_metric': 'rmse'}


In [17]:
num_boost_round = 50
best_model = xgb.train( 
    params,
    dtrain_f,
    num_boost_round=num_boost_round,
)

In [18]:
#best_model.save_model("model_16-04.model")
loaded_model = best_model

In [35]:
y_pred = loaded_model.predict(dtest_e)
y_test = Y_e.values

In [36]:
print('Explained variance : ',explained_variance_score(y_test, y_pred))
print('R2_value : ',r2_score(y_test, y_pred))
print('Mean absolute error MAE :', mean_absolute_error(y_test, y_pred))
print('Root Mean squared error RMSE :', mean_squared_error(y_test, y_pred, squared=False))
print('Median absolute error MAD:', median_absolute_error(y_test, y_pred))

Explained variance :  0.9806574875157648
R2_value :  0.9780011287560375
Mean absolute error MAE : 147.1447723388672
Root Mean squared error RMSE : 179.86605711716666
Median absolute error MAD: 125.6630859375


In [29]:
# Calculate mean absolute percentage error (MAPE)
errors = abs(y_pred-y_test)
mape = 100 * (errors / y_test) # Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('MAX MAPE :', max(mape))
print('MAPE:', round(np.mean(mape), 2), '%.')
print('Accuracy:', round(accuracy, 2), '%.')

MAX MAPE : 14.5605900432834
MAPE: 5.41 %.
Accuracy: 94.59 %.


In [37]:
test_models = X_test_e_model_names.values
test_input_shapes = X_test_e_input_shapes.values

In [38]:
import csv
with open('./measured_vs_predicted_XGBoost_dim1.csv', 'w', newline='') as file : 
    writer = csv.writer(file)
    writer.writerow(['CNN_model', 'input_shape', 'model&shape', 'measured', 'predicted', 'absolute_deviation', 'MAPE (in %)'])
    for i in range(0, len(y_pred)) :
        f = ''+test_models[i]+'_'+test_input_shapes[i]
        writer.writerow([test_models[i], test_input_shapes[i], f, y_test[i], y_pred[i],abs(y_pred[i]-y_test[i]), ((abs(y_pred[i]-y_test[i]))/y_test[i])*100])